# Implementacion el metodo de la maxima verosimilitud para la regresion logistica
Basicamente vamos a pasar toda la parte matematica a python. Esto se puede saltear, ya que existen metodos ya existentes.
### Definir la funcion L(b)

In [2]:
from IPython.display import display, Math, Latex
display(Math(r'L(\beta)=\sum_{i=1}^n P_i^{y_i}(1-Pi)^{y_i}'))

<IPython.core.display.Math object>

In [3]:
# entorno = vector de y sub i y el vector de probabilidades
def likelihood(y, pi):
    import numpy as np
    total_sum = 1
    sum_in = list(range(1, len(y)+1))
    for i in range(len(y)):
        sum_in[i] = np.where(y[i]==1, pi[i], 1-pi[i])
        ##condicion, valor a modificar , modificacion
        total_sum = total_sum * sum_in[i]
    return total_sum

## Calcular la probabilidad para cada observacion

In [4]:
display(Math(r'P_i = P(x_i) = \frac{1}{1+e^{-\sum_{j=0}^k\beta_j\cdot x_{ij}}} '))

<IPython.core.display.Math object>

In [5]:
# x son los conjunto de puntos, y vector betas
def logitprobs(X,beta):
    import numpy as np
    n_rows = np.shape(X)[0] #numeros de filas
    n_cols = np.shape(X)[1] #numero de columnas
    pi=list(range(1,n_rows+1)) #vector de probabilidades
    expon=list(range(1,n_rows+1)) 
    for i in range(n_rows):
        expon[i] = 0
        for j in range(n_cols):
            ex=X[i][j] * beta[j]
            expon[i] = ex + expon[i]
        with np.errstate(divide="ignore", invalid="ignore"): #en caso de haber problemas con las divisiones
            pi[i]=1/(1+np.exp(-expon[i]))
    return pi

### Calcular la matriz diagonal W

In [6]:
display(Math(r'W= diag(P_i \cdot (1-P_i))_{i=1}^n'))

<IPython.core.display.Math object>

In [15]:
# vector de probabilidades p sub i
def findW(pi):
    import numpy as np
    n = len(pi)
    W = np.zeros(n*n).reshape(n,n) #.zeros hace un vector de n ceros y reshape la convierte en una matriz n n
    for i in range(n):
        W[i,i]=pi[i]*(1-pi[i])
        W[i,i].astype(float)
    return W

### Obtener la solución de la función logística
Pasa a traves del metodo Newton Rapson.
* La formula de newton puede no converser nunca, por eso tenemos que tener un limite

In [8]:
display(Math(r"\beta_{n+1} = \beta_n -\frac{f(\beta_n)}{f'(\beta_n)}"))
display(Math(r"f(\beta) = X(Y-P)"))
display(Math(r"f'(\beta) = XWX^T"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [11]:
#x variables de entradas, Y prediccion y Limit es el tamanio del paso, esto quiere decir que se va a seguir
# ejecutando hasta que el paso sea demasiado pequenio
def logistics(X, Y, limit):
    import numpy as np
    from numpy import linalg #para calcular la inversa (la segunda y tercer imagen)
    nrow = np.shape(X)[0] #nmero de filas
    bias = np.ones(nrow).reshape(nrow,1) #creamos un vector de unos con el mismo numero de filas
    X_new = np.append(X, bias, axis = 1) #va a servir para calculos adicionales
    ncol = np.shape(X_new)[1] #n de columnas
    beta = np.zeros(ncol).reshape(ncol,1) #mejorar una columna de ceros. esto se encargara newton
    root_dif = np.array(range(1,ncol+1)).reshape(ncol,1) #aniadimos son las raices del sistema
    iter_i = 10000 
    while(iter_i>limit):
        print("Iter:i"+str(iter_i) + ", limit:" + str(limit))
        pi = logitprobs(X_new, beta) #llamamos nuesta funcion
        print("Pi:"+str(pi))
        W = findW(pi) #buscamos la w
        print("W:"+str(W))
        num = (np.transpose(np.matrix(X_new))*np.matrix(Y - np.transpose(pi)).transpose()) # numerador de Newton
        #np matrix: hacer producto matricial
        #np transpose: para transposer la matriz
        den = (np.matrix(np.transpose(X_new))*np.matrix(W)*np.matrix(X_new)) #denominador de Newton
        root_dif = np.array(linalg.inv(den)*num) #incremento
        beta = beta + root_dif
        print("Beta: "+str(beta))
        iter_i = np.sum(root_dif*root_dif)
        ll = likelihood(Y, pi)
    return beta

### Comprobacion experimental

In [10]:
import numpy as np
X = np.array(range(10)).reshape(10,1)
X

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [13]:
Y = [0,0,0,0,1,0,1,0,1,1]
bias = np.ones(10).reshape(10,1)
x_new = np.append(X,bias,axis=1)
x_new

array([[0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [7., 1.],
       [8., 1.],
       [9., 1.]])

In [16]:
a = logistics(X,Y,0.00001)
# LA MATRIZ W es cada una de las probabilidades 
# pi matriz probabilidad 
# betas las incognitas
# iterr: cuanto nos hemos movidos
# esto termina cuando termina de converger
#Las betas que devuelven son 2:
# la primera es el coeficiente de la variable X
# el segundo es la ordenada al origen

Iter:i10000, limit:1e-05
Pi:[array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5])]
W:[[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25]]
Beta: [[ 0.43636364]
 [-2.36363636]]
Iter:i5.777190082644626, limit:1e-05
Pi:[array([0.08598797]), array([0.12705276]), array([0.18378532]), array([0.2583532]), array([0.35019508]), array([0.45467026]), array([0.56329497]), array([0.66616913]), array([0.75533524]), array([0.82687453])]
W:[[0.07859

Con nuestra funcion lo que nos termina danto es:

**`Y = 0.66220827 - 3.69557172 * x`**
## Con paquetes de python

In [17]:
import statsmodels.api as sm

In [18]:
logit_model = sm.Logit(Y, x_new)

In [19]:
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.431012
         Iterations 6


In [26]:
result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                        Results: Logit
===============================================================
Model:              Logit            Pseudo R-squared: 0.360   
Dependent Variable: y                AIC:              12.6202 
Date:               2020-03-07 21:10 BIC:              13.2254 
No. Observations:   10               Log-Likelihood:   -4.3101 
Df Model:           1                LL-Null:          -6.7301 
Df Residuals:       8                LLR p-value:      0.027807
Converged:          1.0000           Scale:            1.0000  
No. Iterations:     6.0000                                     
-----------------------------------------------------------------
          Coef.    Std.Err.      z      P>|z|     [0.025   0.975]
-----------------------------------------------------------------
x1        0.6622     0.4001    1.6551   0.0979   -0.1220   1.4464
const    -3.6956     2.2889   -1.6145   0.1064   -8.1818   0.7906
===============================================================

"""